Convert a list of image-image Relations to absolute relative image positions

In [1]:
from os.path import abspath
from sys import path
path.insert(0, abspath("../.."))  # Repository directory relative to this file.
from MISalign.model.relation import Relation
from MISalign.model.image import Image
from MISalign.model.mis_file import MisFile

import numpy as np
from matplotlib import pyplot as plt

In [14]:
file_paths=[
    abspath(r"..\..\example\data\set_a\a_myimages01.jpg"),
    abspath(r"..\..\example\data\set_a\a_myimages02.jpg"),
    abspath(r"..\..\example\data\set_a\a_myimages03.jpg"),
    abspath(r"..\..\example\data\set_a\a_myimages04.jpg"),
    abspath(r"..\..\example\data\set_a\a_myimages05.jpg"),
    abspath(r"..\..\example\data\set_a\a_myimages06.jpg"),
    abspath(r"..\..\example\data\set_a\a_myimages07.jpg"),
    abspath(r"..\..\example\data\set_a\a_myimages08.jpg"),
    abspath(r"..\..\example\data\set_a\a_myimages09.jpg"),
    abspath(r"..\..\example\data\set_a\a_myimages10.jpg"),
    ]
mf=MisFile(image_fps=file_paths)

In [15]:
from os.path import split

## Rectangular solve

In [26]:
#TODO image names in misfile - tail from path?
image_names=[split(x)[1] for x in mf.image_fps]
mf._relations=[Relation(a,b,'r',(25*(i+1),30*(i+1))) for i,(a,b) in enumerate(zip(image_names[0:-1],image_names[1:]))]
relations=mf.get_rels('r')
display(relations)
#TODO set/update relations
origin=image_names[0]

[[('a_myimages01.jpg', 'a_myimages02.jpg'), (25, 30)],
 [('a_myimages02.jpg', 'a_myimages03.jpg'), (50, 60)],
 [('a_myimages03.jpg', 'a_myimages04.jpg'), (75, 90)],
 [('a_myimages04.jpg', 'a_myimages05.jpg'), (100, 120)],
 [('a_myimages05.jpg', 'a_myimages06.jpg'), (125, 150)],
 [('a_myimages06.jpg', 'a_myimages07.jpg'), (150, 180)],
 [('a_myimages07.jpg', 'a_myimages08.jpg'), (175, 210)],
 [('a_myimages08.jpg', 'a_myimages09.jpg'), (200, 240)],
 [('a_myimages09.jpg', 'a_myimages10.jpg'), (225, 270)]]

In [28]:
[origin in r[0] for r in relations]

[True, False, False, False, False, False, False, False, False]

In [43]:
# Develop relation map
found=[image_names.index(origin)]
matched=[]
resolved=[]
relation_map=dict({x:[] for x in image_names})

while len(resolved)<len(relations):
    for i in found:
        for ii,x in enumerate(image_names):
            if (ii not in found) & (ii not in resolved) & (ii not in matched):
                i_match=[image_names[i] in r[0] for r in relations]
                ii_match=[image_names[ii] in r[0] for r in relations]
                full_match=[im&iim for im,iim in zip(i_match,ii_match)]
                if any(full_match):
                    relation_map[image_names[i]].append((image_names[ii],relations[full_match.index(True)]))
                    matched.append(ii)
        resolved.append(i)
    found=matched
    matched=[]
    #break if stuck
    if found==[]:
        break

for k,v in relation_map.items():
    print(k)
    for x in v:
        print("   ",x)

a_myimages01.jpg
    ('a_myimages02.jpg', [('a_myimages01.jpg', 'a_myimages02.jpg'), (25, 30)])
a_myimages02.jpg
    ('a_myimages03.jpg', [('a_myimages02.jpg', 'a_myimages03.jpg'), (50, 60)])
a_myimages03.jpg
    ('a_myimages04.jpg', [('a_myimages03.jpg', 'a_myimages04.jpg'), (75, 90)])
a_myimages04.jpg
    ('a_myimages05.jpg', [('a_myimages04.jpg', 'a_myimages05.jpg'), (100, 120)])
a_myimages05.jpg
    ('a_myimages06.jpg', [('a_myimages05.jpg', 'a_myimages06.jpg'), (125, 150)])
a_myimages06.jpg
    ('a_myimages07.jpg', [('a_myimages06.jpg', 'a_myimages07.jpg'), (150, 180)])
a_myimages07.jpg
    ('a_myimages08.jpg', [('a_myimages07.jpg', 'a_myimages08.jpg'), (175, 210)])
a_myimages08.jpg
    ('a_myimages09.jpg', [('a_myimages08.jpg', 'a_myimages09.jpg'), (200, 240)])
a_myimages09.jpg
    ('a_myimages10.jpg', [('a_myimages09.jpg', 'a_myimages10.jpg'), (225, 270)])
a_myimages10.jpg


In [49]:
absolute_position={origin:(0,0)}
solving=[origin]
cansolve=[]
solved=[]
while len(solving)>0:
    for s in solving:
        for image_name,rel in relation_map[s]:
            cansolve.append(image_name)
            if rel[0][0]==s:
                direction=1
            else:
                direction=-1
            absolute_position[image_name]=(absolute_position[s][0]+direction*rel[1][0],absolute_position[s][1]+direction*rel[1][1])
    solved+=solving
    solving=cansolve
    cansolve=[]
display(absolute_position)

{'a_myimages01.jpg': (0, 0),
 'a_myimages02.jpg': (25, 30),
 'a_myimages03.jpg': (75, 90),
 'a_myimages04.jpg': (150, 180),
 'a_myimages05.jpg': (250, 300),
 'a_myimages06.jpg': (375, 450),
 'a_myimages07.jpg': (525, 630),
 'a_myimages08.jpg': (700, 840),
 'a_myimages09.jpg': (900, 1080),
 'a_myimages10.jpg': (1125, 1350)}

In [ ]:
#TODO maxima/minima adjustment for non-purely positive sets
    #TODO Images sizes.